This Colab Notebooks was adpated from already existing code available on [this](https://github.com/odenizgiz/Podcasts-Data) GitHub repository. The goal is to extract the list of popular podcasts on the Apple website in order to have a table containing various metatada, including cover image and textual description.

In [ ]:
%%capture
!pip install pattern3

In [ ]:
import requests
import pattern3.web as web
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
from progressbar import ProgressBar
import pickle

In [ ]:
webpage = 'https://itunes.apple.com/us/genre/podcasts/id26?mt=2'

In [ ]:
def extract_links(webpage):
    try: 
        url = web.URL(webpage)
        bs = BeautifulSoup(url.download(cached = False)) 
        links_page = []
        for link in bs.findAll('a', href=True):
            links_page.append(link['href'])
#             print (link['href'])
    except:
        pass 
    return links_page

In [ ]:
links = extract_links(webpage)
print(links)

['#ac-gn-menustate', '#', '//www.apple.com/', '//www.apple.com/us/shop/goto/bag', '//www.apple.com/', '//www.apple.com/mac/', '//www.apple.com/ipad/', '//www.apple.com/iphone/', '//www.apple.com/watch/', '//www.apple.com/tv/', '//www.apple.com/music/', 'https://support.apple.com', '//www.apple.com/us/search', '//www.apple.com/us/shop/goto/bag', 'http://www.apple.com/itunes/overview/', 'http://www.apple.com/itunes/music/', 'http://www.apple.com/itunes/video/', 'http://www.apple.com/itunes/charts/', '#', 'https://music.apple.com/us/genre/music/id34', 'https://itunes.apple.com/us/genre/movies/id33', 'https://itunes.apple.com/us/genre/tv-shows/id32', 'https://apps.apple.com/us/genre/ios/id36', 'https://podcasts.apple.com/us/genre/podcasts/id26', 'https://books.apple.com/us/genre/books/id38', 'https://music.apple.com/us/genre/music-videos/id31', 'https://books.apple.com/us/genre/audiobooks/id50000024', 'https://podcasts.apple.com/us/genre/podcasts-arts/id1301', 'https://podcasts.apple.com/u

In [ ]:
def detect_patterns(links, pattern):
    pattern_links = [] 
    for link in links:
            if re.search(pattern, link):
                pattern_links.append(link)
                print (link)
    return pattern_links

<br><br/>

### 2. Extract Genre Links 

In [ ]:
pattern = 'https://podcasts.apple.com/us/genre/podcasts-' 

In [ ]:
genre_links = detect_patterns(links, pattern)

https://podcasts.apple.com/us/genre/podcasts-arts/id1301
https://podcasts.apple.com/us/genre/podcasts-arts-books/id1482
https://podcasts.apple.com/us/genre/podcasts-arts-design/id1402
https://podcasts.apple.com/us/genre/podcasts-arts-fashion-beauty/id1459
https://podcasts.apple.com/us/genre/podcasts-arts-food/id1306
https://podcasts.apple.com/us/genre/podcasts-arts-performing-arts/id1405
https://podcasts.apple.com/us/genre/podcasts-arts-visual-arts/id1406
https://podcasts.apple.com/us/genre/podcasts-business/id1321
https://podcasts.apple.com/us/genre/podcasts-business-careers/id1410
https://podcasts.apple.com/us/genre/podcasts-business-entrepreneurship/id1493
https://podcasts.apple.com/us/genre/podcasts-business-investing/id1412
https://podcasts.apple.com/us/genre/podcasts-business-management/id1491
https://podcasts.apple.com/us/genre/podcasts-business-marketing/id1492
https://podcasts.apple.com/us/genre/podcasts-business-non-profit/id1494
https://podcasts.apple.com/us/genre/podcasts-c

### 3. Extract Popular Podcasts 

In [ ]:
pattern2 = 'https://podcasts.apple.com/us/podcast/'

In [ ]:
podcast_links = []

In [ ]:
for genre_link in genre_links:
    all_links = extract_links(genre_link)
    podcast_links.append(detect_patterns(all_links, pattern2))


Streaming output truncated to the last 5000 lines.
https://podcasts.apple.com/us/podcast/mlb-network-intentional-talk/id430939693
https://podcasts.apple.com/us/podcast/mlb-pipeline/id906717198
https://podcasts.apple.com/us/podcast/shea-station-mets-podcast/id1580123422
https://podcasts.apple.com/us/podcast/from-phenom-to-the-farm/id1497327828
https://podcasts.apple.com/us/podcast/ftn-fantasy-baseball-podcast/id1523360049
https://podcasts.apple.com/us/podcast/growing-cubs-a-chicago-prospect-podcast/id1470342753
https://podcasts.apple.com/us/podcast/circling-the-bases-fantasy-baseball/id1112825340
https://podcasts.apple.com/us/podcast/behind-the-braves/id1437194308
https://podcasts.apple.com/us/podcast/d1baseball/id959621656
https://podcasts.apple.com/us/podcast/royals-review-for-kansas-city-royals-fans/id972757794
https://podcasts.apple.com/us/podcast/turning-the-corner-a-show-about-the-detroit/id1557916983
https://podcasts.apple.com/us/podcast/draft-champions-podcast/id1494556568
https

In [ ]:
podcast_links2 = list(set(np.concatenate(podcast_links).tolist()))

In [ ]:
with open('podcast_links', 'wb') as fp:
    pickle.dump(podcast_links2, fp)

In [ ]:
with open ('podcast_links', 'rb') as fp:
    podcast_links2 = pickle.load(fp)


In [ ]:
len(podcast_links2)

23210

<br><BR/>

### 4. Get Names, Descriptions 

In [ ]:
columns= ['authors', 'Name', 'Artwork', 'Genre IDs', 'Primary Genre', 'Episode Count', 'Episode Durations','iTunes URL', 'Feed URL', 'Podcast URL', 'Description']
df_podcast = pd.DataFrame([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], columns = columns) 

In [ ]:
df_podcast

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Get artwork, feedurl genreIDs from the iTunes API: 
def data_from_API(ID):

    url = 'https://itunes.apple.com/lookup?id=' + ID

    try:
        response = requests.get(url) 
        data = response.json()
        
        if data["resultCount"] != 0: 
            for elem in data["results"]:   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                                                #remove two if here
                            if "artworkUrl600" in elem.keys():
                                artwork = elem["artworkUrl600"]
                            if "feedUrl" in elem.keys(): 
                                rss_feed = elem["feedUrl"]
                            if "genres" in elem.keys():
                                genre_IDs = elem["genres"]
                            if 'collectionName' in elem.keys():
                                title = elem['collectionName']
                            if 'artistName' in elem.keys():
                                author = elem['artistName']
                            if 'primaryGenreName' in elem.keys():
                                prim_genre = elem['primaryGenreName']
                                    
                                    
                                return [artwork, rss_feed, genre_IDs, title, author, prim_genre]
                            
                            else: 
                                return None 

                    else:
                        return None 
                          
        else: 
            return None

    except:
        None 

In [ ]:
def get_description(webpage):
    
    global df_podcast
    iTunes_URL = webpage
    podcast_id = re.findall('[0-9]+', re.findall('id[0-9]+', iTunes_URL)[0])[0]
    api_data = data_from_API(podcast_id)
    
    if api_data is not None: 
        artwork = api_data[0]
        feedurl = api_data[1]
        genre_IDs = api_data[2]
        title = api_data[3]
        authors = api_data[4]
        prim_genre = api_data[5]
        
      #i removed some of the try statments here and used the data from the api call
        try: 
            url = web.URL(webpage)
            bs = BeautifulSoup(url.download(cached = False))               
            podcast_link = 0 #not interested in those 3 here, so i set them to 0 and don't search for them
            durations = 0  
            episode_count = 0



            try:              
                description = bs.find('div', 'product-hero-desc') #changed the div in which bs searches the description
                if description is not None:
                    descr_parag = description.findAll('p')
                    for parag in descr_parag:
                        podcast_description = parag.getText()

            except:

                podcast_description = 0 
              


            my_row = [authors, title, artwork, genre_IDs,prim_genre, episode_count, durations, iTunes_URL, feedurl, podcast_link, podcast_description]
            my_row_pd = pd.DataFrame([my_row], columns = columns)  
            df_podcast = df_podcast.append(my_row_pd, ignore_index=True)

            return df_podcast

        except: 
            return None 
        
    else: 
        return None 

In [ ]:
podcast_links3 = list(set(podcast_links2))

In [ ]:
print (len(podcast_links2), len(podcast_links3))

23210 23210


<br> <br/>

In [ ]:
pbar = ProgressBar()

In [ ]:
for link in pbar(podcast_links2):
    get_description(link)

100% (23210 of 23210) |##################| Elapsed Time: 5:25:56 Time:  5:25:56


In [ ]:
df_podcast

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
0,0,0,0,0,0,0,0,0,0,0,0
1,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"[Wilderness, Podcasts, Sports, Leisure, Games]",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed
2,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"[History, Podcasts, Society & Culture]",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill..."
3,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"[Music Commentary, Podcasts, Music]",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ..."
4,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"[Fashion & Beauty, Podcasts, Arts, Health & Fi...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...
...,...,...,...,...,...,...,...,...,...,...,...
23006,"Look Out, Sir!","Look Out, Sir! Warhammer 40k Podcast",https://is5-ssl.mzstatic.com/image/thumb/Podca...,"[Games, Podcasts, Leisure, Hobbies]",Games,0,0,https://podcasts.apple.com/us/podcast/look-out...,https://feeds.buzzsprout.com/195242.rss,0,"The Look Out, Sir! is a fortnightly (sometimes..."
23007,Gen-Z Media | Wondery,Treasure Island 2020,https://is3-ssl.mzstatic.com/image/thumb/Podca...,"[Drama, Podcasts, Fiction]",Drama,0,0,https://podcasts.apple.com/us/podcast/treasure...,https://rss.art19.com/treasure-island-2020,0,James Hawkins helps his mom run a motel in mod...
23008,Maximum Fun,Triple Click,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"[Technology, Podcasts, Society & Culture]",Technology,0,0,https://podcasts.apple.com/us/podcast/triple-c...,https://feeds.simplecast.com/6WD3bDj7,0,Video games: They’re serious business! Except ...
23009,"Bianca A. Solorzano, M.Ed.",Montessori Babies,https://is1-ssl.mzstatic.com/image/thumb/Podca...,"[Education for Kids, Podcasts, Kids & Family]",Education for Kids,0,0,https://podcasts.apple.com/us/podcast/montesso...,https://feeds.buzzsprout.com/1443274.rss,0,Current research shows that the first couple y...


In [ ]:
df_podcast.drop(df_podcast.index[0], inplace=True)
dfpodcast = df_podcast.reset_index(drop=True)

In [ ]:
df_podcast

,authors,Name,Artwork,Genre IDs,Primary Genre,Episode Count,Episode Durations,iTunes URL,Feed URL,Podcast URL,Description
1,Firearms Radio Network,Firearms Radio Network (All Shows),https://is1-ssl.mzstatic.com/image/thumb/Podca...,"[Wilderness, Podcasts, Sports, Leisure, Games]",Wilderness,0,0,https://podcasts.apple.com/us/podcast/firearms...,http://feeds.feedburner.com/FRNetwork,0,Master Feed
2,Jacob Mchangama,Clear and Present Danger - A history of free s...,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"[History, Podcasts, Society & Culture]",History,0,0,https://podcasts.apple.com/us/podcast/clear-an...,https://freespeechhistory.libsyn.com/rss,0,"Why have kings, emperors, and governments kill..."
3,Kyric McMillan and Nick Vassolo,2 Idiots and a List,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"[Music Commentary, Podcasts, Music]",Music Commentary,0,0,https://podcasts.apple.com/us/podcast/2-idiots...,https://anchor.fm/s/4a47e010/podcast/rss,0,"Welcome to “2 Idiots and a List”, the podcast ..."
4,Kate Spencer & Doree Shafrir,Forever35,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"[Fashion & Beauty, Podcasts, Arts, Health & Fi...",Fashion & Beauty,0,0,https://podcasts.apple.com/us/podcast/forever3...,https://rss.acast.com/forever35,0,Which night creams will make you look well-res...
5,Maverick Alexander,DarbyCast,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"[Comedy Fiction, Podcasts, Fiction]",Comedy Fiction,0,0,https://podcasts.apple.com/us/podcast/darbycas...,https://anchor.fm/s/1c855d9c/podcast/rss,0,"Neither the Darbyshire book series, nor this p..."
...,...,...,...,...,...,...,...,...,...,...,...
23006,"Look Out, Sir!","Look Out, Sir! Warhammer 40k Podcast",https://is5-ssl.mzstatic.com/image/thumb/Podca...,"[Games, Podcasts, Leisure, Hobbies]",Games,0,0,https://podcasts.apple.com/us/podcast/look-out...,https://feeds.buzzsprout.com/195242.rss,0,"The Look Out, Sir! is a fortnightly (sometimes..."
23007,Gen-Z Media | Wondery,Treasure Island 2020,https://is3-ssl.mzstatic.com/image/thumb/Podca...,"[Drama, Podcasts, Fiction]",Drama,0,0,https://podcasts.apple.com/us/podcast/treasure...,https://rss.art19.com/treasure-island-2020,0,James Hawkins helps his mom run a motel in mod...
23008,Maximum Fun,Triple Click,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"[Technology, Podcasts, Society & Culture]",Technology,0,0,https://podcasts.apple.com/us/podcast/triple-c...,https://feeds.simplecast.com/6WD3bDj7,0,Video games: They’re serious business! Except ...
23009,"Bianca A. Solorzano, M.Ed.",Montessori Babies,https://is1-ssl.mzstatic.com/image/thumb/Podca...,"[Education for Kids, Podcasts, Kids & Family]",Education for Kids,0,0,https://podcasts.apple.com/us/podcast/montesso...,https://feeds.buzzsprout.com/1443274.rss,0,Current research shows that the first couple y...


In [ ]:
dfpodcast.to_csv("df_popular_podcasts.csv", index=False)